#### Global & Local Modules

In [1]:
import os
from itertools import permutations 
import sys
import numpy as np
import scipy.sparse as sp
from tqdm import tqdm
from keras.callbacks import EarlyStopping
from keras.layers import Input, Dropout
from keras.models import Model
from keras.optimizers import Adam
from keras.regularizers import l2
from rgcn import GaussianGraphConvolution, kl_reg

Using TensorFlow backend.


#### Vendor Modules

In [2]:
# the path of execution
EXE_PATH = os.path.abspath(os.path.curdir)
# the path of the vendor files
VENDOR_PATH = os.path.join(EXE_PATH, 'vendor')
# the vendors to include in the system path
VENDORS = ['keras-gcn']
# create the absolute paths for all vendors
VENDORS = list(map(lambda x: os.path.join(VENDOR_PATH, x), VENDORS))
# update the Python path to include necessary vendor module
sys.path += VENDORS
# import vendor modules
from kegra.layers.graph import GraphConvolution
from kegra.utils import load_data, preprocess_adj, get_splits

# Data

In [3]:
def attack_edges(attack_ratio: float) -> 'callable':
    """
    Create an edge attack function with given attack ratio.

    Args:
        attack_ratio: the ratio of noise to clean edges

    Returns:
        a callable function for attacking a graph

    """
    def _attack_edges(edges: np.ndarray) -> np.ndarray:
        f"""
        Attack the edges of the given list of edges.

        Args:
            edges: the ndarray of tuples representing edges to attack

        Returns:
            a new ndarray of edges with random edges added

        Note:
            the ratio of noise to clean edges is {attack_ratio}

        """
        # create the set of all possible edges
        all_edges = set(permutations(list(range(edges.max())), 2))
        # convert the ndarray of edges to a set of tuples
        edges = set(map(tuple, edges))
        # create the set of edges to sample by subtracting the existing
        # edges from the set of possible edges
        sample_edges = all_edges - edges
        # select random edges without replacement based on the noise ratio
        attack_edges = np.random.choice(list(range(len(sample_edges))), 
            size=int(len(edges) * attack_ratio), 
            replace=False)
        attack_edges = [edge for (i, edge) in enumerate(list(sample_edges)) if i in attack_edges]
        # create the new ndarray of edges with the attack edges added
        edges = list(edges) + attack_edges
        return np.array(edges)
    return _attack_edges

In [4]:
X, A, y = load_data('data/cora/', dataset='cora', attack_edges=attack_edges(0))
X /= X.sum(1).reshape(-1, 1)
A = preprocess_adj(A)
y_train, y_val, y_test, idx_train, idx_val, idx_test, train_mask, val_mask, test_mask = get_splits(y)

In [5]:
X

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [6]:
A

<2708x2708 sparse matrix of type '<class 'numpy.float64'>'
	with 13264 stored elements in Compressed Sparse Row format>

In [7]:
y

array([[0, 1, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 1, 0, ..., 0, 0, 0]], dtype=int32)

# Model

## Input Layers

In [8]:
X_in = Input(shape=(X.shape[1], ), name='features')
X_in

W1209 16:28:34.382604 140292071569216 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



<tf.Tensor 'features:0' shape=(?, 1433) dtype=float32>

In [9]:
A_in = Input(shape=(None, None), batch_shape=(None, None), sparse=True, name='graph')
A_in

W1209 16:28:34.394092 140292071569216 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:515: The name tf.sparse_placeholder is deprecated. Please use tf.compat.v1.sparse_placeholder instead.



## Training Loop

In [10]:
def train(model, epochs=200, patience=10):
    """
    Train the given model.

    Args:
        model: the model to train
        epochs: the maximum number of training epochs
        patience: the number of patience epochs for early stoping

    Returns:
        a tuple of the testing loss and accuracy

    """
    history = model.fit([X, A], y_train,
        sample_weight=train_mask,
        batch_size=A.shape[0],
        epochs=epochs,
        shuffle=False,
        callbacks=[
            EarlyStopping(monitor='val_loss', patience=patience),
        ],
        validation_data = ([X, A], y_val, val_mask)
    )
    test = model.evaluate([X, A], y_test, 
        sample_weight=test_mask, 
        batch_size=A.shape[0],
    )
    return test

## GCN (baseline)

In [11]:
# create the layers
H = Dropout(0.5)(X_in)
H = GraphConvolution(32, 1, 
    activation='relu', 
    kernel_regularizer=l2(5e-4)
)([H, A_in])
H = Dropout(0.5)(H)
Y = GraphConvolution(y.shape[1], 1, 
    activation='softmax'
)([H, A_in])
# create the model
gcn = Model(inputs=[X_in, A_in], outputs=Y)
gcn.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.01), weighted_metrics=['acc'])
gcn.summary()

W1209 16:28:34.407257 140292071569216 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1209 16:28:34.408177 140292071569216 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W1209 16:28:34.412121 140292071569216 deprecation.py:506] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W1209 16:28:34.420112 140292071569216 deprecation_wrapper.py:119]

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
features (InputLayer)           (None, 1433)         0                                            
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 1433)         0           features[0][0]                   
__________________________________________________________________________________________________
graph (InputLayer)              (None, None)         0                                            
__________________________________________________________________________________________________
graph_convolution_1 (GraphConvo (None, 32)           45888       dropout_1[0][0]                  
                                                                 graph[0][0]                      
__________

In [12]:
train(gcn)[1]

W1209 16:28:34.534081 140292071569216 deprecation.py:323] From /home/bitcommander/.local/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 2708 samples, validate on 2708 samples
Epoch 1/200
2708/2708 [==============================] - 1s 343us/step - loss: 1.9777 - weighted_acc: 0.1286 - val_loss: 1.9540 - val_weighted_acc: 0.3800
Epoch 2/200
2708/2708 [==============================] - 0s 7us/step - loss: 1.9525 - weighted_acc: 0.3857 - val_loss: 1.9319 - val_weighted_acc: 0.3600
Epoch 3/200
2708/2708 [==============================] - 0s 7us/step - loss: 1.9300 - weighted_acc: 0.3143 - val_loss: 1.9112 - val_weighted_acc: 0.3633
Epoch 4/200
2708/2708 [==============================] - 0s 8us/step - loss: 1.9058 - weighted_acc: 0.3857 - val_loss: 1.8923 - val_weighted_acc: 0.3767
Epoch 5/200
2708/2708 [==============================] - 0s 6us/step - loss: 1.8816 - weighted_acc: 0.4071 - val_loss: 1.8748 - val_weighted_acc: 0.3933
Epoch 6/200
2708/2708 [==============================] - 0s 8us/step - loss: 1.8586 - weighted_acc: 0.4000 - val_loss: 1.8586 - val_weighted_acc: 0.4033
Epoch 7/200
2708/2708 [=========

Epoch 54/200
2708/2708 [==============================] - 0s 7us/step - loss: 1.3203 - weighted_acc: 0.7429 - val_loss: 1.4699 - val_weighted_acc: 0.6900
Epoch 55/200
2708/2708 [==============================] - 0s 7us/step - loss: 1.3430 - weighted_acc: 0.7143 - val_loss: 1.4626 - val_weighted_acc: 0.6933
Epoch 56/200
2708/2708 [==============================] - 0s 7us/step - loss: 1.2835 - weighted_acc: 0.7357 - val_loss: 1.4560 - val_weighted_acc: 0.7000
Epoch 57/200
2708/2708 [==============================] - 0s 7us/step - loss: 1.2479 - weighted_acc: 0.8071 - val_loss: 1.4504 - val_weighted_acc: 0.7067
Epoch 58/200
2708/2708 [==============================] - 0s 7us/step - loss: 1.2764 - weighted_acc: 0.7786 - val_loss: 1.4457 - val_weighted_acc: 0.7067
Epoch 59/200
2708/2708 [==============================] - 0s 7us/step - loss: 1.2359 - weighted_acc: 0.7929 - val_loss: 1.4401 - val_weighted_acc: 0.7100
Epoch 60/200
2708/2708 [==============================] - 0s 7us/step - loss

Epoch 108/200
2708/2708 [==============================] - 0s 7us/step - loss: 0.9475 - weighted_acc: 0.8857 - val_loss: 1.1687 - val_weighted_acc: 0.8000
Epoch 109/200
2708/2708 [==============================] - 0s 9us/step - loss: 0.8752 - weighted_acc: 0.9000 - val_loss: 1.1674 - val_weighted_acc: 0.7967
Epoch 110/200
2708/2708 [==============================] - 0s 9us/step - loss: 0.9497 - weighted_acc: 0.8714 - val_loss: 1.1655 - val_weighted_acc: 0.8000
Epoch 111/200
2708/2708 [==============================] - 0s 7us/step - loss: 0.8906 - weighted_acc: 0.9071 - val_loss: 1.1628 - val_weighted_acc: 0.7933
Epoch 112/200
2708/2708 [==============================] - 0s 8us/step - loss: 0.8981 - weighted_acc: 0.8714 - val_loss: 1.1589 - val_weighted_acc: 0.7933
Epoch 113/200
2708/2708 [==============================] - 0s 8us/step - loss: 0.9156 - weighted_acc: 0.8714 - val_loss: 1.1548 - val_weighted_acc: 0.7933
Epoch 114/200
2708/2708 [==============================] - 0s 8us/step

Epoch 161/200
2708/2708 [==============================] - 0s 7us/step - loss: 0.7692 - weighted_acc: 0.9286 - val_loss: 1.0396 - val_weighted_acc: 0.8133
Epoch 162/200
2708/2708 [==============================] - 0s 7us/step - loss: 0.7602 - weighted_acc: 0.9214 - val_loss: 1.0423 - val_weighted_acc: 0.8100
Epoch 163/200
2708/2708 [==============================] - 0s 7us/step - loss: 0.7383 - weighted_acc: 0.9571 - val_loss: 1.0417 - val_weighted_acc: 0.8067
Epoch 164/200
2708/2708 [==============================] - 0s 7us/step - loss: 0.7558 - weighted_acc: 0.9286 - val_loss: 1.0383 - val_weighted_acc: 0.8100
Epoch 165/200
2708/2708 [==============================] - 0s 7us/step - loss: 0.7532 - weighted_acc: 0.9214 - val_loss: 1.0336 - val_weighted_acc: 0.8233
Epoch 166/200
2708/2708 [==============================] - 0s 7us/step - loss: 0.7303 - weighted_acc: 0.9357 - val_loss: 1.0288 - val_weighted_acc: 0.8133
Epoch 167/200
2708/2708 [==============================] - 0s 7us/step

0.8159999251365662

## Gaussian Graph Convolution

In [13]:
# create the layers
H = Dropout(0.6)(X_in)
M1, S1 = GaussianGraphConvolution(16,
    is_first=True,
    dropout=0.6
)([H, A_in])
Y = GaussianGraphConvolution(y.shape[1],
    is_last=True,
    last_activation='softmax',
    mean_regularizer=None,
    variance_regularizer=None
)([M1, S1, A_in])
# create the model
rgcn = Model(inputs=[X_in, A_in], outputs=Y)
rgcn.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.01), weighted_metrics=['acc'])
rgcn.add_loss(kl_reg(M1, S1, 5e-4), [M1, S1])
rgcn.summary()

W1209 16:28:39.618536 140292071569216 nn_ops.py:4224] Large dropout rate: 0.6 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.
W1209 16:28:39.642402 140292071569216 nn_ops.py:4224] Large dropout rate: 0.6 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.
W1209 16:28:39.649566 140292071569216 nn_ops.py:4224] Large dropout rate: 0.6 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.
W1209 16:28:39.670408 140292071569216 deprecation.py:323] From /home/bitcommander/Documents/robust-graph-convolutional-networks-against-adversarial-attacks-implementation/rgcn/ggc.py:206: Normal.__init__ (from tensorflow.python.ops.distributions.normal) is deprecated and will be removed after 2019-01-01.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.co

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
features (InputLayer)           (None, 1433)         0                                            
__________________________________________________________________________________________________
dropout_3 (Dropout)             (None, 1433)         0           features[0][0]                   
__________________________________________________________________________________________________
graph (InputLayer)              (None, None)         0                                            
__________________________________________________________________________________________________
gaussian_graph_convolution_1 (G [(None, 16), (None,  45856       dropout_3[0][0]                  
                                                                 graph[0][0]                      
__________

In [14]:
train(rgcn)[1]

Train on 2708 samples, validate on 2708 samples
Epoch 1/200
2708/2708 [==============================] - 0s 103us/step - loss: 1.9745 - weighted_acc: 0.1571 - val_loss: 1.9798 - val_weighted_acc: 0.1500
Epoch 2/200
2708/2708 [==============================] - 0s 8us/step - loss: 1.9652 - weighted_acc: 0.1214 - val_loss: 1.9535 - val_weighted_acc: 0.1667
Epoch 3/200
2708/2708 [==============================] - 0s 7us/step - loss: 1.9450 - weighted_acc: 0.1571 - val_loss: 1.9504 - val_weighted_acc: 0.1867
Epoch 4/200
2708/2708 [==============================] - 0s 8us/step - loss: 1.9264 - weighted_acc: 0.2643 - val_loss: 1.9497 - val_weighted_acc: 0.1700
Epoch 5/200
2708/2708 [==============================] - 0s 8us/step - loss: 1.9200 - weighted_acc: 0.2143 - val_loss: 1.9349 - val_weighted_acc: 0.2100
Epoch 6/200
2708/2708 [==============================] - 0s 8us/step - loss: 1.9314 - weighted_acc: 0.1929 - val_loss: 1.9281 - val_weighted_acc: 0.2267
Epoch 7/200
2708/2708 [=========

Epoch 54/200
2708/2708 [==============================] - 0s 7us/step - loss: 1.4195 - weighted_acc: 0.7786 - val_loss: 1.5735 - val_weighted_acc: 0.7400
Epoch 55/200
2708/2708 [==============================] - 0s 7us/step - loss: 1.3998 - weighted_acc: 0.8143 - val_loss: 1.5598 - val_weighted_acc: 0.7267
Epoch 56/200
2708/2708 [==============================] - 0s 8us/step - loss: 1.3722 - weighted_acc: 0.8429 - val_loss: 1.5626 - val_weighted_acc: 0.7700
Epoch 57/200
2708/2708 [==============================] - 0s 8us/step - loss: 1.3577 - weighted_acc: 0.8214 - val_loss: 1.5512 - val_weighted_acc: 0.7367
Epoch 58/200
2708/2708 [==============================] - 0s 7us/step - loss: 1.3827 - weighted_acc: 0.8071 - val_loss: 1.5334 - val_weighted_acc: 0.7500
Epoch 59/200
2708/2708 [==============================] - 0s 7us/step - loss: 1.3471 - weighted_acc: 0.8286 - val_loss: 1.5178 - val_weighted_acc: 0.7500
Epoch 60/200
2708/2708 [==============================] - 0s 7us/step - loss

Epoch 108/200
2708/2708 [==============================] - 0s 8us/step - loss: 1.0253 - weighted_acc: 0.8857 - val_loss: 1.2837 - val_weighted_acc: 0.8167
Epoch 109/200
2708/2708 [==============================] - 0s 7us/step - loss: 1.0626 - weighted_acc: 0.8786 - val_loss: 1.2607 - val_weighted_acc: 0.8167
Epoch 110/200
2708/2708 [==============================] - 0s 7us/step - loss: 1.0135 - weighted_acc: 0.9143 - val_loss: 1.2733 - val_weighted_acc: 0.8000
Epoch 111/200
2708/2708 [==============================] - 0s 8us/step - loss: 1.0306 - weighted_acc: 0.9071 - val_loss: 1.2508 - val_weighted_acc: 0.8167
Epoch 112/200
2708/2708 [==============================] - 0s 8us/step - loss: 1.0093 - weighted_acc: 0.9071 - val_loss: 1.2609 - val_weighted_acc: 0.8000
Epoch 113/200
2708/2708 [==============================] - 0s 7us/step - loss: 1.0685 - weighted_acc: 0.8571 - val_loss: 1.2753 - val_weighted_acc: 0.8067
Epoch 114/200
2708/2708 [==============================] - 0s 7us/step

Epoch 161/200
2708/2708 [==============================] - 0s 8us/step - loss: 0.8517 - weighted_acc: 0.9429 - val_loss: 1.1653 - val_weighted_acc: 0.8133
Epoch 162/200
2708/2708 [==============================] - 0s 8us/step - loss: 0.8658 - weighted_acc: 0.9429 - val_loss: 1.1893 - val_weighted_acc: 0.8000
Epoch 163/200
2708/2708 [==============================] - 0s 8us/step - loss: 0.8175 - weighted_acc: 0.9143 - val_loss: 1.1310 - val_weighted_acc: 0.8133
Epoch 164/200
2708/2708 [==============================] - 0s 7us/step - loss: 0.8975 - weighted_acc: 0.9286 - val_loss: 1.1265 - val_weighted_acc: 0.8267
Epoch 165/200
2708/2708 [==============================] - 0s 7us/step - loss: 0.8639 - weighted_acc: 0.9214 - val_loss: 1.1369 - val_weighted_acc: 0.8133
Epoch 166/200
2708/2708 [==============================] - 0s 7us/step - loss: 0.8890 - weighted_acc: 0.9143 - val_loss: 1.1213 - val_weighted_acc: 0.8300
Epoch 167/200
2708/2708 [==============================] - 0s 7us/step

0.8229999542236328